## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sqlite3
from database import Database

In [ ]:
db = Database("sql_terugkom_ochtend.db")

## Shared vs Unique Properties

In [ ]:
# Product table with common properties
# Note: properties as columns
db.query("DROP TABLE IF EXISTS Products;")

db.query("""
    CREATE TABLE Products (
        id INTEGER,
        name TEXT,
        brand TEXT,
        description TEXT
    );
""")

db.query("""
    INSERT INTO Products
    VALUES
        (1, "Samsung SSD 1TB", "Samsung", "Samsung - Solid State Disk - 1 TB - Laadt je data nog sneller!"),
        (2, "Samsung Monitor 24 inch", "Samsung", "Samsung Monitor - 24 inch - Supergoed scherm voor data scientist!")
    ;
""")

db.query("SELECT * FROM Products")

In [ ]:
# ProductProperties with unique properties
# Note: properties as rows
db.query("DROP TABLE IF EXISTS ProductProperties;")
db.query("""
    CREATE TABLE ProductProperties (
        product_id INTEGER,
        label TEXT,
        value TEXT
    );
""")

db.query("""
    INSERT INTO ProductProperties
    VALUES
        (1, "Opslagruimte", "1 TB"),
        (1, "Form factor", "2.5 inch"),
        (1, "Interface", "SATA-600"),
        (1, "Snelheid - Lezen", "3.400 MB/s"),
        (1, "Snelheid - Schrijven", "2.300 MB/s")
""")

db.query("SELECT * FROM ProductProperties")

In [ ]:
# Pivot in Pandas (because SQLite doesn't support it...)
(
    db.query("SELECT * FROM ProductProperties")
    .pivot(index="product_id", columns="label", values="value")
)

In [ ]:
"""
WITH temp_sales AS (
    SELECT
        country,
        cyear,
        sales
    FROM dbo.sales_country_year
)
SELECT
    country,
    [2012],
    [2013],
    [2014]
FROM temp_sales
PIVOT (
    SUM(sales)
    FOR cyear IN ([2012], [2013], [2014])
) AS Pivot;
"""

In [ ]:
# Clean up
db.query("DROP TABLE IF EXISTS Products;")
db.query("DROP TABLE IF EXISTS ProductProperties;")

## Transactions

Use transactions to "bundle" queries together, allowing you to roll back changes if one subquery fails.

In [ ]:
db.query("DROP TABLE IF EXISTS Products;")
db.query("""
    CREATE TABLE Products (
        id INTEGER PRIMARY KEY,
        name TEXT UNIQUE,
        brand TEXT,
        description TEXT
    );
""")

In [ ]:
db.query("DROP TABLE IF EXISTS ProductProperties;")
db.query("""
    CREATE TABLE ProductProperties (
        product_id INTEGER,
        label TEXT,
        value TEXT
    );
""")

In [ ]:
db.list_tables()

In [ ]:
# Set up connection for execute script
conn = sqlite3.connect("sql_terugkom_ochtend.db")
cursor = conn.cursor()

In [ ]:
try:
    
    cursor.executescript("""
        BEGIN TRANSACTION;

        INSERT INTO ProductProperties (product_id, label, value)
        VALUES
            (1, 'Test Property A', 1),
            (1, 'Test Property B', 2),
            (1, 'Test Property C', 3)
        ;

        INSERT INTO Products (name, brand, description)
            VALUES('Test Product', 'Testing', 'Some test product');

        COMMIT;
    """)
    print("All done!")

except sqlite3.IntegrityError:
    
    print("OH NO! Some constraint failed...")
    cursor.execute("ROLLBACK;")

In [ ]:
db.query("SELECT * FROM Products;")

In [ ]:
db.query("SELECT * FROM ProductProperties;")

In [ ]:
# Clean up
db.query("DROP TABLE IF EXISTS Products;")
db.query("DROP TABLE IF EXISTS ProductProperties;")

## Upsert: Insert or Update

Upsert will try to insert a record, if a conflict occurs it will try to update the existing record.
The upsert operation only touches the conflicting row (unlike a regular update statement).


**Note: Not part of the SQL standard, each RDBMS will have its own implementation!**

In [ ]:
db.query("DROP TABLE IF EXISTS Products;")
db.query("""
    CREATE TABLE Products (
        id INTEGER PRIMARY KEY,
        name TEXT UNIQUE,
        brand TEXT,
        description TEXT,
        version INTEGER DEFAULT 0
    );
""")

In [ ]:
# Normal insert
# Note: Fails after first run due to UNIQUE constraint on name
db.query("""
    INSERT INTO Products (name, brand, description)
        VALUES('Test Product', 'Testing', 'Some test product')
""")

In [ ]:
db.query("SELECT * FROM Products;")

In [ ]:
# Use "upsert" to update records
db.query("""
    INSERT INTO Products (name, brand, description)
        VALUES('Test Product', 'Testing', 'Some updated test product')
    ON CONFLICT (name) DO UPDATE SET
        name = excluded.name,
        brand = excluded.brand,
        description = excluded.description,
        version = version + 1
    ;
""")

In [ ]:
db.query("SELECT * FROM Products;")

In [ ]:
# Clean up
db.query("DROP TABLE IF EXISTS Products;")

## Recursive queries

In [ ]:
db.query("DROP TABLE IF EXISTS Categories;")
db.query("""
    CREATE TABLE Categories (
        id INTEGER,
        parent INTEGER,
        label TEXT
    );
""")

In [ ]:
db.query("""
    INSERT INTO Categories (id, parent, label)
    VALUES
        (1, 0, 'Computers'),
        (2, 1, 'Harde schijven'),
        (3, 2, 'SSD'),
        (4, 2, 'HDD'),
        (5, 1, 'Monitoren'),
        (6, 0, 'Witgoed'),
        (7, 6, 'Wasmachines')
    ;
""")

In [ ]:
db.query("SELECT * FROM Categories;")

**Recursive query format:**

In [ ]:
# Items under main menu item
category = "Computers"
category = "Witgoed"

db.query(f"""
    WITH RECURSIVE under_item (id, label, level) AS (
            SELECT
                id,
                label,
                0
            FROM Categories
            WHERE label = '{category}'
        
        UNION ALL
        
        SELECT
            cat.id,
            cat.label,
            under_item.level + 1
        FROM Categories cat
        JOIN under_item ON cat.parent=under_item.id
    )
    SELECT * FROM under_item ORDER BY id
    ;
""")

In [ ]:
# Reverse query: Bread crumb trail
item = "SSD"
# item = "Monitoren"

db.query(f"""
    WITH RECURSIVE above_item (id, parent, label, level) AS (
            SELECT
                id,
                parent,
                label,
                0
            FROM Categories
            WHERE label = '{item}'
        
        UNION ALL
        
        SELECT
            cat.id,
            cat.parent,
            cat.label,
            above_item.level - 1
        FROM Categories cat
        JOIN above_item ON cat.id=above_item.parent
            AND cat.id >= 0
    )
    SELECT *
    FROM above_item
    ORDER BY level
    ;
""")

In [ ]:
# Clean up
db.query("DROP TABLE IF EXISTS Categories;")